# Estimating Baseline Performance
기준 성능을 측정하는 건 모델 평가에 적절한 metric을 정하는 것만큼 중요하다. 이 노트북에선 영화 추천 시나리오의 두 가지 예 : 1.평점 예측과 2. top-k 추천을 다룬다.

## Why dose baseline performance matter?
**기준 성능**은 모델에 기대하는 최소한의 성능 혹은 모델 비교를 위한 시작점이다. 모델을 훈련하고 검증 결과를 볼 때 metrics를 어떻게 해석해야 할지에 대한 것이나 훈련된 모델이 간단한 rule-based 모델보다 좋은지 이해하는 데 도움을 준다. 모델 훈련이 잘 안됐거나 적절한 metric이 아니면 NDCG가 더 낮게 나오는 것...

## How can we estimate the baseline performance?
기준 성능을 측정하려면 먼저 기준 모델을 선정하고 메인 모델에 사용할 metric을 이용해 평가한다. 일반적으로 simple-rule 혹은 심지어는 zero-rule(regression이나 분류 모델의 평균 예측) 모델이면 기준 모델로 충분하다. 이미 수행중인 모델이 있고, 성능을 향상시키려는 경우엔 모델의 이전 결과로도 충분하다.   
중요한건, 서로 다른 문제 혹은 비즈니스 목표엔 서로 다른 기준 접근법이 필요하다는 것이다.   
   
이 노트에서 영화 추천을 위한 기준 성능을 어떻게 측정할 지 살펴본다. 평점 예측 방법을 위해선 평균을 사용한다. 즉, 기준 모델은 유저의 평점 예측을 위해 유저가 이전에 다른 영화에 매긴 평점들의 평균을 사용한다. top-k 추천 방법을 위해선 가장 많이 평가된 top-k 영화를 기준 모델로 삼는다. We choose the number of ratings here because we regard the binary signal of 'rated vs. not-rated' as user's implicit preference when evaluating ranking metrics.

In [1]:
import sys
import itertools
import pandas as pd

from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.datasets.pandas_df_utils import filter_by
from recommenders.evaluation.python_evaluation import rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, recall_at_k

In [ ]:
data = movielens.load_pandas_df(size='100k', header=['UserId', 'MovieId', 'Rating','Timestamp'])

In [12]:
tt = pd.DataFrame(data, columns=['UserId','MovieId','Rating','Timestamp'])
tt

,UserId,MovieId,Rating,Timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
...,...,...,...,...
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156


In [13]:
tt.to_csv('Movielens_100k.csv', header=['UserId','MovieId','Rating','Timestamp'], index=False)

In [80]:
data = pd.read_csv('Movielens_100k.csv')
train, test = python_random_split(data, ratio=0.75, seed=42)

## 1. Rating prediction baseline


In [81]:
# Calculate avg ratings from the training set
user_ratings = train.groupby(['UserId'])['Rating'].mean()
user_ratings = user_ratings.to_frame().reset_index()
user_ratings.rename(columns={'Rating':'AvgRating'}, inplace=True)

In [82]:
# Generate prediction for the test set
baseline_pred = pd.merge(test, user_ratings, on=['UserId'], how='inner')
baseline_pred['AvgRating']# = str(baseline_pred['AvgRating'])
baseline_pred.loc[baseline_pred['UserId']==1].head()

,UserId,MovieId,Rating,Timestamp,AvgRating
12215,1,233,2.0,878542552,3.69697
12216,1,159,3.0,875073180,3.69697
12217,1,238,4.0,875072235,3.69697
12218,1,100,5.0,878543541,3.69697
12219,1,63,2.0,878543196,3.69697


문제 1. `MovieId`의 datatype이 달라서 비교 안됨.   
문제 2. `MovieId`에 `None`이 있어서 int로 변경 불가.   
-> `outer`가 아니라 `inner` merge 모두 해결

In [83]:
# Evaluate how baseline model will perform on regression metrics
baseline_pred = baseline_pred[['UserId','MovieId','AvgRating']]
# baseline_pred = baseline_pred.astype({'MovieId':'int'})
cols = {'col_user':'UserId', 'col_item':'MovieId',
       'col_rating':'Rating', 'col_prediction':'AvgRating'}

In [84]:
eval_rmse = rmse(test, baseline_pred, **cols)
eval_mae = mae(test, baseline_pred, **cols)
eval_rsq = rsquared(test, baseline_pred, **cols)
eval_exp = exp_var(test, baseline_pred, **cols)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsq,
      "exp var:\t%f" % eval_exp, sep='\n')

RMSE:		1.044885
MAE:		0.836925
rsquared:	0.136491
exp var:	0.136496


## 2. Top-k recommendation baseline
**가장 인기있는** 아이템을 추천하는 것은 많은 추천 시나리오에 먹히는 직관적이고 간단한 방법이다.

In [85]:
item_cnt = train['MovieId'].value_counts().to_frame().reset_index()
item_cnt.columns = ['MovieId', 'Count']
item_cnt.head()

,MovieId,Count
0,50,419
1,181,382
2,100,381
3,258,377
4,288,371


In [86]:
user_item_col = ['UserId', 'MovieId']

test_users = test['UserId'].unique()
user_item_list = list(itertools.product(test_users, item_cnt['MovieId']))
user_item = pd.DataFrame(user_item_list, columns=user_item_col)
print('Number of user-item pairs:', len(user_item))

# Remove seen items
user_item_unseen = filter_by(user_item, train, user_item_col)
print('After remove seen items:', len(user_item_unseen))

Number of user-item pairs: 1546764
After remove seen items: 1471784


In [89]:
# Generate recommendations
baseline_recom = pd.merge(item_cnt, user_item_unseen, on=['MovieId'], how='inner')
baseline_recom.head()

,MovieId,Count,UserId
0,50,419,877
1,50,419,815
2,50,419,416
3,50,419,259
4,50,419,598


In [91]:
cols['col_prediction'] = 'Count'

eval_map = map_at_k(test, baseline_recom, k=10, **cols)
eval_ndcg = ndcg_at_k(test, baseline_recom, k=10, **cols)
eval_precision = precision_at_k(test, baseline_recom, k=10, **cols)
eval_recall = recall_at_k(test, baseline_recom, k=10, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG@K:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.055008
NDCG@K:	0.252867
Precision@K:	0.224628
Recall@K:	0.111736


In [93]:
if is_jupyter():
    import papermill as pm
    import scrapbook as sb
    sb.glue("map", eval_map)
    sb.glue("ndcg", eval_ndcg)
    sb.glue("precision", eval_precision)
    sb.glue("recall", eval_recall)
    sb.glue("rmse", eval_rmse)
    sb.glue("mae", eval_mae)
    sb.glue("exp_var", eval_exp)
    sb.glue("rsquared", eval_rsq)